## PHYS 105A:  Introduction to Scientific Computing

# Random Numbers and Monte Carlo Methods

Chi-kwan Chan

* In physical science, students very often start with the concept that everything can be done exactly and deterministically.

* This can be one of the biggest misconcept!

* Many physical processes are non-deterministic by nature.  Examples include:
  * Radioactive decay
  * Quantum mechanics

* Sometime, even when the governing equaitons are deterministic, the results are still non-deterministic.  Examples include:
  * 3-body problem
  * Brownian motion
  * Thermal noise

* Therefore, in computer we need some way to model these non-deterministic systems.

* For electronic computers, all operations are deterministic.

* Nevertheless, we can approximate random processes by *pseudo* random number generators.

* These pseudo random number generators can then be used to model non-deterministic systems.

* In addition, people started to realize, even for deterministic problems, randomizing them can still be a very powerful numerical method!  Applications include
  * Numerical integration of high-dimensional space
  * Statistical inference

* This results a large number of random number based numerical methods.

* Monte Carlo is an area of Monaco well known for its world-famous Place du Casino.

* Monte Carlo methods are used to refer to random number based numerical methods.

![Monte Carlo](https://upload.wikimedia.org/wikipedia/commons/1/1b/Monaco_pano.jpg)

In [ ]:
# In order to understand the concept of a random number generator, let's implement one ourself.

mynext = 1

def myrand(): # NOT RECOMMENDED for real application.
    global mynext
    mynext = mynext * 1103515245 + 12345
    return (mynext//65536) % 32768

# This random number generator would generate integers in the domain [0, 32768).
# This random is usually provided to user by

MYRAND_MAX = 32768-1

# There are reasons for choosing the strange constants.  Take a look at
# https://en.wikipedia.org/wiki/Linear_congruential_generator
# if you are interested.

In [ ]:
# Now, every time we run `rand()`, we will get a different number

myrand()

In [ ]:
# For we may just print many of them at the same time:

Rs = [myrand() for i in range(100)]
print(Rs)

In [ ]:
# We may even plot the random numbers

from matplotlib import pyplot as plt

plt.imshow([[myrand() for i in range(100)] for j in range(100)])

In [ ]:
# Sometime it is useful to make sure your random number sequence remains the same.
# In our case, you may notice that we can simply reset the `mynext` global variable to reset the sequence.
# The value you put in `mynext` is often called the "seed".

print('The following two lists are not the same:')
print([myrand() for i in range(10)])
print([myrand() for i in range(10)])

print('We may ensure that they are the same by "seeding" the random number generator with a fixed value:')

mynext = 1234
print([myrand() for i in range(10)])

mynext = 1234
print([myrand() for i in range(10)])

* The above random number generator is very simple and is the *sample* implementation in many ANSI C libraries!

* However, because how the standard was written, this create a lot problems.
  * The standard only require RAND_MAX be at least 32767.  If one want to evulate 1e6 points (which is pretty small, as we will see below), you will actually be evaluating the same 32768 points 30 times each!
  * Some implementation "tried" to imporve the algorithm, e.g., swapping the lower and higher bytes.  But these tricks sometime ruins the generator!
  * We mentioned that integrating high-dimension space is an important application of Monte Carlo methods.  However, the above random number generator create correlation in k-space.
  
* Thankfully, `ptyhon`'s random number generator is based on the "more reliable" [Mersenne Twister algorithm](https://en.wikipedia.org/wiki/Mersenne_Twister).

* From now on, unless for demostration purpose, we will use python's built-in random number generators.

In [ ]:
# Let's now try python's random number library

import random as rnd

print(rnd.random())             # return a random float in the range [0,1)
print(rnd.randrange(100))       # return a random int in the range [0, stop)
print(rnd.randint(a=0,b=99))    # return a random int in the range [a, b+1)
print(rnd.gauss(mu=0, sigma=1)) # sample from a Gaussian distribution

In [ ]:
# We may plot the results of these random number generators

Rs = [rnd.random() for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.randrange(100) for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.gauss(0, 1) for i in range(1000)]
plt.hist(Rs)

In [ ]:
# There is also a seed() function

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])

* Once we have a (pseudo) random number generator, we are ready to develop Monte Carlo methods!

* We will start with a simple example of random walk.  The model is very simple:
  * We start with a (drunk) person at the center of the street.
  * As the person step forward toward +t, the person random also step left +1 or right -1.
  * The problem is, after n steps, how far away is the person away from the center of the street?

In [ ]:
# We may step up this problem in the following way:

T = range(1, 1000+1)
X = [0] # initial position

for t in T:
    last = X[-1]            # last position
    r    = rnd.randint(0,1) # we generate 0 or 1 randomly
    if r == 0:              # depending on r, we step left or right
        curr = last + 1
    else:
        curr = last - 1
    X.append(curr)          # append the current position to the list X

# We may plot this random walk
plt.plot(X)

In [ ]:
# Awesome!
# But in order to find out how random walk behave statistically,
# we want to be able to run many simulations!

# It is convenient to define a function

def randomwalk(n_steps=1000):
    X = [0] # initial position
    
    for t in range(n_steps):
        last = X[-1]            # last position
        r    = rnd.randint(0,1) # we generate 0 or 1 randomly
        if r == 0:              # depending on r, we step left or right
            curr = last + 1
        else:
            curr = last - 1    
        X.append(curr)          # append the current position to the list X
        
    return X                    # return the result

# And we can use this function in another loop.
for i in range(10):
    plt.plot(randomwalk())

In [ ]:
# We may now ask how far away the peron would walk depending on the number of stpes.

D = []
for t in T:
    X = randomwalk(t)
    D.append(abs(X[-1]))
    
plt.plot(D)

# Clearly, the distance gets farther when the number of steps increase.
# But this figure is too noise to read off the dependency.

In [ ]:
# There are multiple ways to make the above figure less noise.
# One way is to simply do multiple numerical experiments for the same number of steps.
# And obtain the average distance.

n_trials = 100
D = []
for t in T:
    M = 0
    for trial in range(n_trials):
        X  = randomwalk(t)
        M += abs(X[-1])
    M /= n_trials
    D.append(M)
    
plt.plot(D)

# The plot is much better!

In [ ]:
# Later in the class, we will learn how to fit a curve.
# But for now, let's simply plot this in log-log scale.
# And compare it with the law of diffusion D ~ sqrt(T)

plt.loglog(T, D)
plt.plot(T, [t**0.5 for t in T])

* You may use this simple random walk model to model real physical process.

* For example, the Brownian motion, which describe how pollen is randomly pushed by water molecules.

![Brownian motion](https://upload.wikimedia.org/wikipedia/commons/c/c2/Brownian_motion_large.gif)

* Einstein published a paper on Brownian motion in 1905, which is one of his first major scientific contributions.

In [ ]:
# The simplest model of Brownian motion is simply a two-dimension random walk.

X = randomwalk()
Y = randomwalk()

plt.plot(X, Y)

# The resulting plot looks slightly funny because random walk forces x and y both to move at exactly one step.
# The final outcome is that the particle can only move in diagonal directions.

In [ ]:
# But this artifact becomes irrelevant when we model random walk for many many more steps.

X = randomwalk(100000)
Y = randomwalk(100000)

plt.plot(X, Y)